In [1]:
import os
import cprv1.cprv1 as cpr
import numpy as np
import pandas as pd
import datetime
codigo = 99
self = cpr.Nivel(codigo=codigo,user='sample_user',passwd='s@mple_p@ss',SimuBasin=True)

self.GetGeo_Cell_Basics()
self.GetGeo_Parameters()
self.table = 'estaciones_estaciones'
fields = self.read_sql('describe %s'%self.table)['Field'].values
df = pd.DataFrame(cpr.info.ESTACIONES).T
df.columns = ['label','siata_name']
siata_table = df['siata_name'].dropna()
remote = cpr.SqlDb(**cpr.info.REMOTE)
REDES = cpr.info.REDES
format = str(REDES)[1:-1]
CODIGOS = remote.read_sql("SELECT Codigo FROM estaciones WHERE Red in (%s) and estado = 'A'"%format)['Codigo']
CODIGOS = str(np.array(CODIGOS.values,str))[1:-1].replace('\n','').replace("' '","','")
FIELDS = ','.join(siata_table.values)
consulta = remote.read_sql("SELECT %s FROM estaciones WHERE Codigo in (%s)"%(FIELDS,CODIGOS))
consulta[consulta==-999]=np.NaN
consulta[consulta.isnull()]=np.NaN
upload = pd.DataFrame(index = consulta.index,columns = df.index)
df['local'] = df.index
consulta.columns = df.set_index('siata_name').loc[consulta.columns,'local'].values
upload[consulta.columns]=consulta
offset_old = upload['offset_old']
offset_old[offset_old==0.0] = np.NaN
upload['offset_old'] = offset_old
now = datetime.datetime.now().strftime('%Y-%m-%d')
upload[['timestamp','updated']] = now
upload['vel_sup'] = 0
upload['slug'] = map(lambda x:x.replace(' ','').replace('.','').replace('-','').replace('(','').replace(')',''),upload['nombre'].values)
upload['user_id'] = 1
self.table = 'estaciones_estaciones'
upload['slug']=upload['codigo'].astype(str)+'-'+upload['slug']

def insert_data(self,fields,values,*keys,**kwargs):
    '''
    inserts data into SQL table from list of fields and values
    Parameters
    ----------
    fields   = list of fields names from SQL db
    values   = list of values to be inserted
    Example
    -------
    insert_data(['fecha','nivel'],['2017-07-13',0.5])
    '''
    values = str(list(values)).strip('[]')
    fields = str(list(fields)).strip('[]').replace("'","")
    execution = 'INSERT INTO %s (%s) VALUES (%s)'%(self.table,fields,values)
    return execution
upload['flag_modelo_wmf']='0'
upload['codigo']  = upload['codigo'].astype(int)

upload.loc[upload[upload['codigo']==99].index[0],'flag_modelo_wmf'] = 1
codigos_nivel = remote.read_sql("select Codigo,NombreEstacion,red from estaciones where estado = 'A' and (red='nivel' or red='mocoa-nivel')")['Codigo'].values
codigos_nivel=np.array(codigos_nivel,int)
serie = pd.Series(upload['codigo'].index,index=upload['codigo'].values).loc[codigos_nivel]
# estaciones con velocidad superficial
upload.loc[serie.loc[[99,92,106,140]].values,'vel_sup']=[157,176,177,185]
upload.loc[serie.loc[codigos_nivel].values,'clase'] = 'Nivel'
query = "SELECT Codigo FROM estaciones where red in ('Pluviografica','mocoa-pluvio','pluviografica') and estado = 'A'"
codigos_pluvio = np.array(remote.read_sql(query)['Codigo'].values,int)
serie = pd.Series(upload['codigo'].index,index=upload['codigo'].values).loc[codigos_pluvio]
upload.loc[serie.loc[codigos_pluvio].values,'clase'] = 'Pluvio'
serie = pd.Series(upload['codigo'].index,index=upload['codigo'].values).loc[codigos_nivel]
basins_main_path = '/media/nicolas/maso/Mario/basins/'
upload.loc[serie.values,'nc_path'] = map(lambda x: '%s%s.nc'%(basins_main_path,x),np.array(serie.index,str))
for shape in ['polygon','net','stream']:
    shape_path = "/media/nicolas/maso/Mario/shapes/%s/"%shape
    upload.loc[serie.values,'%s_path'%shape] = map(lambda x: '%s%s/%s'%(shape_path,x,x),np.array(serie.index,str))
for count,codigo in enumerate(codigos_nivel): 
    try:
        self = cpr.Nivel(codigo=codigo,user='sample_user',passwd='s@mple_p@ss',SimuBasin=True)
        self.GetGeo_Cell_Basics()
        self.GetGeo_Parameters()
        df = pd.DataFrame(pd.Series(self.GeoParameters))
        df.loc[cpr.info.GEOPARAMETERS.keys(),'field'] = cpr.info.GEOPARAMETERS.values()
        df = df.set_index('field')[0]
        df = df[df.index.dropna()]
        upload.loc[serie.loc[self.codigo],df.index] = df.values
    except:
        print 'failed:%s'%codigo
self.table = 'estaciones_estaciones'
codigos = [ 128,108,245,109,106,186,124,135,247,140,96,101,246,92,94,245,238,251,1014,
            1013,260,158,182,93,239,90,104,143,183,240,99,91,115,116,134,152,166,179,
            155,236,173,178,196,195,259,268,98,272,273 ]

x_sensor = [ 8.0,4.23,11.66,3.5,17.0,5.75,12.6,3.08,3.0,24.0,4.2,0.8,4.2,1.3,12.11,11.66,
            12.54,4.1,6.88,8.74,21.0,3.87,2.45,31.17,6.4,11.6,2.55,4.66,2.8,1.5,21.0,18.95,
            1.55,8.21,2.5,3.0,2.0,8.0,5.5,14.0,2.0,1.5,29.8,3.92,2.42,4.4,5.0,5.74,3.18 ]
@property
def offsetData(self):
    '''Tabla historico_bancallena_offset'''
    df = self.mysql_query('select * from historico_bancallena_offset;')
    df.columns = self.mysql_query('describe historico_bancallena_offset;')[0].values
    df[df==-9999]=np.NaN
    return df.set_index('codigo')

print 'stations without x_sensor'
for codigo in codigos_nivel:
    self = cpr.Nivel(codigo=codigo,user='sample_user',passwd='s@mple_p@ss')
    upload.loc[serie[codigo],'offset'] = self.offset_remote()
    try:
        self.x_sensor = cpr.info.X_SENSOR[self.codigo]
        upload.loc[serie[codigo],'x_sensor'] = self.x_sensor
    except:
        print self.codigo
    upload.loc[serie[codigo],'sirena'] = '0'
pictures_main_path = '/media/nicolas/maso/Mario/pics/'

def tostrf(x):
    try:
        value = x.strftime('%Y-%m-%d')
    except:
        value = np.NaN
    return value
upload['fecha_instalacion'] = upload['fecha_instalacion'].apply(lambda x:tostrf(x))
# making directories
for codigo in codigos_nivel:
    ruta_folder = pictures_main_path+str(codigo)
    os.system('mkdir %s'%ruta_folder)
    upload.loc[serie[codigo],'pic_path'] = ruta_folder
    
self.table = 'estaciones_estaciones'


failed:1013
failed:1014
failed:195
failed:273
stations without x_sensor
145
150
169
161
181
187
192
265


In [2]:
upload = upload.drop('usr',axis=1)

In [3]:
upload.to_csv('/media/nicolas/maso/Mario/migration_%s.csv'%datetime.datetime.now().strftime('%Y%m%d%H%M'))

In [6]:
index = upload.fillna('NULL').index
for count,ids in enumerate(index):
    print('%.2f %%,%s out of %s'%((count+1)*100.0/len(index),count+1,len(index)))
    serie = upload.fillna('NULL').loc[ids]
    serie.index = map(lambda x:x.encode("utf-8"),serie.index)
    execution = insert_data(self,list(serie.index),list(serie.values)).replace("'NULL'","NULL")
    self.execute_sql(execution)
    
#upload.to_csv('/media/nicolas/maso/Mario/migration_%s.csv'%datetime.datetime.now().strftime('%Y%m%d%H%M'))

0.51 %,1 out of 195
1.03 %,2 out of 195
1.54 %,3 out of 195
2.05 %,4 out of 195
2.56 %,5 out of 195
3.08 %,6 out of 195
3.59 %,7 out of 195
4.10 %,8 out of 195
4.62 %,9 out of 195
5.13 %,10 out of 195
5.64 %,11 out of 195
6.15 %,12 out of 195
6.67 %,13 out of 195
7.18 %,14 out of 195
7.69 %,15 out of 195
8.21 %,16 out of 195
8.72 %,17 out of 195
9.23 %,18 out of 195
9.74 %,19 out of 195
10.26 %,20 out of 195
10.77 %,21 out of 195
11.28 %,22 out of 195
11.79 %,23 out of 195
12.31 %,24 out of 195
12.82 %,25 out of 195
13.33 %,26 out of 195
13.85 %,27 out of 195
14.36 %,28 out of 195
14.87 %,29 out of 195
15.38 %,30 out of 195
15.90 %,31 out of 195
16.41 %,32 out of 195
16.92 %,33 out of 195
17.44 %,34 out of 195
17.95 %,35 out of 195
18.46 %,36 out of 195
18.97 %,37 out of 195
19.49 %,38 out of 195
20.00 %,39 out of 195
20.51 %,40 out of 195
21.03 %,41 out of 195
21.54 %,42 out of 195
22.05 %,43 out of 195
22.56 %,44 out of 195
23.08 %,45 out of 195
23.59 %,46 out of 195
24.10 %,47 out o